# Pandas Tutorial from RealPython

My worksheet from the tutorial from RealPython:

[Pandas Tutorial](https://realpython.com/fast-flexible-pandas/)

Excellent Tutorial on how operate on a dataframe effciently and how to store a preprocessed dataframe with the HDFStore


In [64]:
import pandas as pd
print(pd.__version__)

0.23.4


In [65]:
df = pd.read_csv('./data/demand_profile.csv')

In [66]:
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [67]:
# note that date_time is of type 'object' which means string, and not a datetime type
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [68]:
# re-write the date_time columns as a datetime type
%timeit df['date_time'] = pd.to_datetime(df['date_time'])

421 µs ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
df.dtypes

date_time     datetime64[ns]
energy_kwh           float64
dtype: object

In [70]:
# notice how the contents are now visually different because of the data type change
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


Time repeated calls to convert to datetime

In [71]:
df = pd.read_csv('./data/demand_profile.csv')

In [72]:
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [73]:
from timer import timeit

In [74]:
# @timeit, run 3 trials, each trial 10 iterations and display best average
@timeit(repeat=3, number=10)
def convert_to_datetime(df, column_name):
    return pd.to_datetime(df[column_name])


In [75]:
df['date_time'] = convert_to_datetime(df, 'date_time')

Best of 3 trials with 10 function calls per trial:
Function `convert_to_datetime` ran in average of 0.959 seconds.



In [76]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


Change te convert method to use a format specification on the *to_datetime* call.  Telling Pandas was the format is of the incoming datestring speeds up the process considerably.


In [77]:
# @timeit, run 3 trials, each trial 10 iterations and display best average
@timeit(repeat=3, number=10)
def convert_to_datetime_with_format(df, column_name):
    return pd.to_datetime(df[column_name], format='%d/%m/%y %H:%M')

In [78]:
df = pd.read_csv('./data/demand_profile.csv')
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [79]:
df['date_time'] = convert_to_datetime_with_format(df, 'date_time')

Best of 3 trials with 10 function calls per trial:
Function `convert_to_datetime_with_format` ran in average of 0.026 seconds.



In [80]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


During reading of the csv file, we can tell pandas to try to convert the column

In [81]:
%%timeit 
df = pd.read_csv('./data/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'])

1.03 s ± 46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%%timeit 
df = pd.read_csv('./data/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format="%d/%m/%y %H:%M")

35.2 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [83]:
%timeit df = pd.read_csv('./data/demand_profile.csv', parse_dates=['date_time'], infer_datetime_format=True)

1.09 s ± 89.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
df.dtypes

date_time     datetime64[ns]
energy_kwh           float64
dtype: object

In [85]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


## Looping over the Pandas DataFrame

In [86]:
"""
Tarrif Type  Cents.   Time Range
Peak.         28.        17:00-24:00
Shoulder.     20.        7:00 - 17:00
Off-Peak.     12.        0:00 - 7:00
"""
def apply_tariff(kwh, hour):
    """Calculates cost of electricity for given hour."""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

A very bad way to loop through all of the records in a python for loop.

In [87]:
# NOTE: Don't do this!
@timeit(repeat=1, number=2)
def apply_tariff_loop(df):
    """Calculate costs in loop.  Modifies `df` inplace."""
    energy_cost_list = []
    for i in range(len(df)):
        # Get electricity used and hour of day
        energy_used = df.iloc[i]['energy_kwh']
        hour = df.iloc[i]['date_time'].hour
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list

apply_tariff_loop(df)

Best of 1 trials with 2 function calls per trial:
Function `apply_tariff_loop` ran in average of 3.499 seconds.



Use Pandas *iterrows* and *itertuples*

In [88]:
@timeit(repeat=1, number=2)
def apply_tariff_iterrows(df):
    energy_cost_list = []
    for index, row in df.iterrows():
        # Get electricity used and hour of day
        energy_used = row['energy_kwh']
        hour = row['date_time'].hour
        # Append cost list
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list

apply_tariff_iterrows(df)

Best of 1 trials with 2 function calls per trial:
Function `apply_tariff_iterrows` ran in average of 0.865 seconds.



Use Pandas *.apply* method

In [89]:
@timeit(repeat=3, number=100)
def apply_tariff_withapply(df):
    df['cost_cents'] = df.apply(
        lambda row: apply_tariff(
            kwh=row['energy_kwh'],
            hour=row['date_time'].hour),
        axis=1)

apply_tariff_withapply(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_withapply` ran in average of 0.219 seconds.



Even with *.apply* the operation on average is still not significantly faster.  

2.89 seconds the really bad way

0.685 seconds using iterrows

0.215 seconds using the .apply method

But we can do better.

## Vectorized Pandas Operations

When you broadcast you are running a vectorized operation e.g.  df['x'] = df['y'] + 20

With no loops, or apply methods it is considered a vectorized operation and these are fast.  So how can we apply vector operations to the above problem

First, set the date_time column as the index so we can efficiently segment the dataframe into the 3 tariff rate groups.


In [90]:
df = pd.read_csv('./data/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format="%d/%m/%y %H:%M")

In [91]:
df.set_index('date_time', inplace=True)

In [92]:
df.head()

,energy_kwh
date_time,
2013-01-01 00:00:00,0.586
2013-01-01 01:00:00,0.580
2013-01-01 02:00:00,0.572
2013-01-01 03:00:00,0.596
2013-01-01 04:00:00,0.592


In [93]:
@timeit(repeat=3, number=100)
def apply_tariff_isin(df):
    # Define hour range Boolean arrays
    peak_hours = df.index.hour.isin(range(17, 24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    off_peak_hours = df.index.hour.isin(range(0, 7))

    # Apply tariffs to hour ranges
    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12

In [94]:
apply_tariff_isin(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_isin` ran in average of 0.004 seconds.



Now by creating ranges of the dataframe to apply the calculation to significantly increases the performance of the calculation.

In [95]:
df.head()

,energy_kwh,cost_cents
date_time,,
2013-01-01 00:00:00,0.586,7.032
2013-01-01 01:00:00,0.580,6.960
2013-01-01 02:00:00,0.572,6.864
2013-01-01 03:00:00,0.596,7.152
2013-01-01 04:00:00,0.592,7.104


## Pandas *cut* method for better performance

In [98]:
df = pd.read_csv('./data/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format="%d/%m/%y %H:%M")
df.set_index('date_time', inplace=True)

In [99]:
cents_per_kwh = pd.cut(x=df.index.hour,
                           bins=[0, 7, 17, 24],
                           include_lowest=True,
                           labels=[12, 20, 28]).astype(int)

*cents_per_kwh* is not an array of cents for each bin the hour
falls into.  This array can then be used to apply to 
the entire dataset.


In [101]:
cents_per_kwh

array([12, 12, 12, ..., 28, 28, 28])

In [102]:
@timeit(repeat=3, number=100)
def apply_tariff_cut(df):
    cents_per_kwh = pd.cut(x=df.index.hour,
                           bins=[0, 7, 17, 24],
                           include_lowest=True,
                           labels=[12, 20, 28]).astype(int)
    df['cost_cents'] = cents_per_kwh * df['energy_kwh']

In [103]:
apply_tariff_cut(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_cut` ran in average of 0.002 seconds.



In [104]:
df.head()

,energy_kwh,cost_cents
date_time,,
2013-01-01 00:00:00,0.586,7.032
2013-01-01 01:00:00,0.580,6.960
2013-01-01 02:00:00,0.572,6.864
2013-01-01 03:00:00,0.596,7.152
2013-01-01 04:00:00,0.592,7.104


Using cut to create an array of multiplier values, and then broadcasting that over the entire dataframe is the most efficient way to handle the calculation.

## Using numpy

In [109]:
import numpy as np

In [106]:
df = pd.read_csv('./data/demand_profile.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format="%d/%m/%y %H:%M")
df.set_index('date_time', inplace=True)

In [107]:
@timeit(repeat=3, number=100)
def apply_tariff_digitize(df):
    prices = np.array([12, 20, 28])
    bins = np.digitize(df.index.hour.values, bins=[7, 17, 24])
    df['cost_cents'] = prices[bins] * df['energy_kwh'].values

In [110]:
apply_tariff_digitize(df)

Best of 3 trials with 100 function calls per trial:
Function `apply_tariff_digitize` ran in average of 0.001 seconds.



In [111]:
df.head()

,energy_kwh,cost_cents
date_time,,
2013-01-01 00:00:00,0.586,7.032
2013-01-01 01:00:00,0.580,6.960
2013-01-01 02:00:00,0.572,6.864
2013-01-01 03:00:00,0.596,7.152
2013-01-01 04:00:00,0.592,7.104


# Pandas HDF5 File Format

Pandas has a built-in solution for this which uses HDF5 , a high-performance storage format designed specifically for storing tabular arrays of data. Pandas’ HDFStore class allows you to store your DataFrame in an HDF5 file so that it can be accessed efficiently, while still retaining column types and other metadata. It is a dictionary-like class, so you can read and write just as you would for a Python dict object.

## Install tables:

*pip install --upgrade tables*

## Create a HDFStore object for Pandas

In [117]:
df.dtypes

energy_kwh    float64
cost_cents    float64
dtype: object

In [118]:
df.describe()

,energy_kwh,cost_cents
count,8760.000000,8760.000000
mean,0.653600,13.118321
std,0.453193,10.114679
min,0.000000,0.000000
25%,0.285000,5.058000
50%,0.609000,11.000000
75%,0.941000,19.620000
max,3.832000,76.640000


In [113]:
# Create storage object with filename `processed_data`
data_store = pd.HDFStore('processed_data.h5')

# Put DataFrame into the object setting the key as 'preprocessed_df'
data_store['preprocessed_df'] = df
data_store.close()

## Read the Pandas Dataframe from the HDFStore

In [114]:
# Access data store
data_store = pd.HDFStore('processed_data.h5')

# Retrieve data using key
preprocessed_df = data_store['preprocessed_df']
data_store.close()

In [115]:
preprocessed_df.head()

,energy_kwh,cost_cents
date_time,,
2013-01-01 00:00:00,0.586,7.032
2013-01-01 01:00:00,0.580,6.960
2013-01-01 02:00:00,0.572,6.864
2013-01-01 03:00:00,0.596,7.152
2013-01-01 04:00:00,0.592,7.104


In [116]:
preprocessed_df.dtypes

energy_kwh    float64
cost_cents    float64
dtype: object

In [120]:
preprocessed_df.describe()

,energy_kwh,cost_cents
count,8760.000000,8760.000000
mean,0.653600,13.118321
std,0.453193,10.114679
min,0.000000,0.000000
25%,0.285000,5.058000
50%,0.609000,11.000000
75%,0.941000,19.620000
max,3.832000,76.640000
